In [1]:
import pandas as pd
import yfinance as yf
import funzioni as fx
import plotly.offline as pyo
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.metrics import Precision, Recall, AUC
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping
from tensorflow.keras.models import load_model
from sklearn.model_selection import StratifiedKFold
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import os

pd.set_option("display.max_columns", None)

def visualizza_grafico(ticker):
    import plotly.graph_objs as go
    fig = fx.grafico_base(ticker)
    tg = go.Scatter(
        x = ticker[ticker['Target'] == True].index,
        y = ticker[ticker['Target'] == True]['Close'],
        mode = 'markers',
        marker = dict(
            size = 15,
            color = 'rgba(0, 250, 0, .9)'
        ),
        name = 'target'
    )
    fig.add_trace(tg, row=1, col=1)
    pyo.plot(fig, filename="prova.html", auto_open=True)

def addestramento(model, features, target):    
    X, Y = fx.converti_in_XY(features, target)
    Y = np.where(Y, 1, 0)
    class_weights = compute_class_weight('balanced', classes=np.unique(Y), y=Y)
    class_weight_dict = {0: class_weights[0], 1: class_weights[1]}
    print(class_weights) 

    early_stopping = EarlyStopping(monitor='val_loss', patience=10)

    if model == 0:
        model = keras.Sequential([
            layers.LSTM(200, input_shape=(X.shape[1], X.shape[2]), return_sequences=True),
            layers.Dropout(0.2),
            layers.LSTM(200, return_sequences=True),  # Nuovo layer LSTM 1
            layers.Dropout(0.2),
            layers.LSTM(200, return_sequences=True),  # Nuovo layer LSTM 2
            layers.Dropout(0.2),
            layers.LSTM(200),  # Ultimo layer LSTM
            layers.Dropout(0.2),
            layers.Dense(1, activation='sigmoid')
        ])
    custom_optimizer = Adam(learning_rate=0.0001)

    model.compile(
        optimizer=custom_optimizer,
        loss='binary_crossentropy',
        metrics=[
            Precision(name='precision'),
            Recall(name='recall'),
            AUC(name='auc'),
            'accuracy'
        ]
    )

    scaler = StandardScaler()
    n_samples, n_timesteps, n_features = X.shape
    X_reshape = X.reshape((n_samples, n_timesteps * n_features))
    X_scaled = scaler.fit_transform(X_reshape)
    X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.2, random_state=42)
    X_train_scaled = X_train.reshape((-1, n_timesteps, n_features))
    X_test_scaled = X_test.reshape((-1, n_timesteps, n_features))
    
    model.fit(
        X_train_scaled, Y_train,
        epochs=100,
        batch_size=32,
        class_weight=class_weight_dict,
        validation_data=(X_test_scaled, Y_test),
        callbacks=[
            early_stopping
        ]
    )

    loss, precision, recall, auc, accuracy = model.evaluate(X_test_scaled, Y_test)
    print(f"\033[42mLoss: {loss}, Precision: {precision}, Recall: {recall}, AUC: {auc}, Accuracy: {accuracy}\033[0m")
    with open("result.txt", "a") as f:
        f.write(f"Loss: {loss}, Precision: {precision}, Recall: {recall}, AUC: {auc}, Accuracy: {accuracy}\n")
    return model, scaler



In [2]:
elenco_usin = pd.read_excel("DeGiro con formule.xlsx")
elenco_usin = elenco_usin.loc[elenco_usin["Categoria"] != "D", ["USIN", "Ticker"]]
elenco_usin = elenco_usin.sample(frac=1)
tot_usin = len(elenco_usin)
i = 1
if os.path.exists("LSTM.keras"):
    model = load_model(nome_file)
else:
    model = 0
for index, usin in elenco_usin.iterrows():       
    print(f"\r{i} di {tot_usin}) Caricamento ticker {usin['Ticker']}                           ", end=' ', flush=True)
    ticker = yf.download(usin["USIN"], start='2005-01-01', end='2023-07-31', progress=False)
    if len(ticker) < 500:
        continue
    else:
        ticker = fx.crea_indicatori(ticker)
        features = ticker.drop(columns=["Open", "High", "Low", "Close", "Adj Close", "Max_High_Futuro_20d", "Min_Low_Futuro_20d",
                                        "Drawdown_20d", "Perc_Max_High_Futuro_20d", "Perc_Drawdown_20d", "MaxMinRel", "guadagno_max_perc"
                                       ])        
        target = (ticker["Perc_Max_High_Futuro_20d"] >= 20) & (ticker["Perc_Drawdown_20d"] <=5)
        ticker["Target"] = target
        model, scaler = addestramento(model, features=features, target=target) 
        model.save("LSTM.keras")
        i += 1

print("fine addestramento")

1 di 2581) Caricamento ticker SSTI                            [0.61082474 2.75581395]
Epoch 1/100
36/36 [==============================] - 20s 168ms/step - loss: 0.6845 - precision: 0.2417 - recall: 0.5971 - auc: 0.6420 - accuracy: 0.5875 - val_loss: 0.6775 - val_precision: 0.2463 - val_recall: 0.6346 - val_auc: 0.6746 - val_accuracy: 0.5789
Epoch 2/100
36/36 [==============================] - 2s 56ms/step - loss: 0.6494 - precision: 0.2597 - recall: 0.6505 - auc: 0.6766 - accuracy: 0.6007 - val_loss: 0.6429 - val_precision: 0.2403 - val_recall: 0.5962 - val_auc: 0.6829 - val_accuracy: 0.5825
Epoch 3/100
36/36 [==============================] - 2s 53ms/step - loss: 0.6239 - precision: 0.2692 - recall: 0.6456 - auc: 0.7027 - accuracy: 0.6183 - val_loss: 0.6066 - val_precision: 0.2566 - val_recall: 0.5577 - val_auc: 0.7108 - val_accuracy: 0.6246
Epoch 4/100
36/36 [==============================] - 2s 52ms/step - loss: 0.6049 - precision: 0.2902 - recall: 0.6214 - auc: 0.7274 - accuracy: 

Epoch 33/100
36/36 [==============================] - 2s 60ms/step - loss: 0.3610 - precision: 0.4486 - recall: 0.9320 - auc: 0.9003 - accuracy: 0.7801 - val_loss: 0.4173 - val_precision: 0.4272 - val_recall: 0.8462 - val_auc: 0.8750 - val_accuracy: 0.7649
Epoch 34/100
36/36 [==============================] - 2s 59ms/step - loss: 0.3650 - precision: 0.4432 - recall: 0.9272 - auc: 0.8981 - accuracy: 0.7757 - val_loss: 0.4320 - val_precision: 0.4336 - val_recall: 0.9423 - val_auc: 0.8733 - val_accuracy: 0.7649
Epoch 35/100
36/36 [==============================] - 2s 54ms/step - loss: 0.3632 - precision: 0.4563 - recall: 0.9126 - auc: 0.9048 - accuracy: 0.7872 - val_loss: 0.4260 - val_precision: 0.4182 - val_recall: 0.8846 - val_auc: 0.8833 - val_accuracy: 0.7544
Epoch 36/100
36/36 [==============================] - 2s 55ms/step - loss: 0.3545 - precision: 0.4462 - recall: 0.9466 - auc: 0.9100 - accuracy: 0.7775 - val_loss: 0.3987 - val_precision: 0.4490 - val_recall: 0.8462 - val_auc: 0.

Epoch 65/100
36/36 [==============================] - 2s 57ms/step - loss: 0.2606 - precision: 0.5600 - recall: 0.9515 - auc: 0.9518 - accuracy: 0.8558 - val_loss: 0.3553 - val_precision: 0.5634 - val_recall: 0.7692 - val_auc: 0.9059 - val_accuracy: 0.8491
Epoch 66/100
36/36 [==============================] - 2s 54ms/step - loss: 0.2685 - precision: 0.5652 - recall: 0.9466 - auc: 0.9476 - accuracy: 0.8584 - val_loss: 0.3857 - val_precision: 0.5844 - val_recall: 0.8654 - val_auc: 0.9017 - val_accuracy: 0.8632
Epoch 67/100
36/36 [==============================] - 2s 60ms/step - loss: 0.2581 - precision: 0.5490 - recall: 0.9515 - auc: 0.9509 - accuracy: 0.8496 - val_loss: 0.3627 - val_precision: 0.6056 - val_recall: 0.8269 - val_auc: 0.9128 - val_accuracy: 0.8702
Epoch 68/100
36/36 [==============================] - 2s 56ms/step - loss: 0.2597 - precision: 0.5861 - recall: 0.9417 - auc: 0.9471 - accuracy: 0.8690 - val_loss: 0.3666 - val_precision: 0.5844 - val_recall: 0.8654 - val_auc: 0.

40/40 [==============================] - 2s 57ms/step - loss: 0.2505 - precision: 0.2186 - recall: 0.9400 - auc: 0.9622 - accuracy: 0.8641 - val_loss: 0.2370 - val_precision: 0.2051 - val_recall: 1.0000 - val_auc: 0.9857 - val_accuracy: 0.9016
Epoch 15/100
40/40 [==============================] - 2s 53ms/step - loss: 0.2377 - precision: 0.2400 - recall: 0.9600 - auc: 0.9648 - accuracy: 0.8776 - val_loss: 0.2209 - val_precision: 0.2105 - val_recall: 1.0000 - val_auc: 0.9855 - val_accuracy: 0.9048
Epoch 16/100
40/40 [==============================] - 2s 54ms/step - loss: 0.2233 - precision: 0.2500 - recall: 0.9400 - auc: 0.9697 - accuracy: 0.8855 - val_loss: 0.2082 - val_precision: 0.2222 - val_recall: 1.0000 - val_auc: 0.9872 - val_accuracy: 0.9111
Epoch 17/100
40/40 [==============================] - 2s 53ms/step - loss: 0.2165 - precision: 0.2487 - recall: 0.9400 - auc: 0.9709 - accuracy: 0.8847 - val_loss: 0.2097 - val_precision: 0.2222 - val_recall: 1.0000 - val_auc: 0.9872 - val_ac

Epoch 46/100
40/40 [==============================] - 2s 53ms/step - loss: 0.0803 - precision: 0.4630 - recall: 1.0000 - auc: 0.9932 - accuracy: 0.9539 - val_loss: 0.0800 - val_precision: 0.4444 - val_recall: 1.0000 - val_auc: 0.9937 - val_accuracy: 0.9683
Epoch 47/100
40/40 [==============================] - 2s 54ms/step - loss: 0.1126 - precision: 0.4016 - recall: 0.9800 - auc: 0.9870 - accuracy: 0.9412 - val_loss: 0.1666 - val_precision: 0.3478 - val_recall: 1.0000 - val_auc: 0.9839 - val_accuracy: 0.9524
Epoch 48/100
40/40 [==============================] - 2s 56ms/step - loss: 0.1189 - precision: 0.4261 - recall: 0.9800 - auc: 0.9865 - accuracy: 0.9467 - val_loss: 0.2988 - val_precision: 0.2400 - val_recall: 0.7500 - val_auc: 0.9676 - val_accuracy: 0.9333
Epoch 49/100
40/40 [==============================] - 2s 53ms/step - loss: 0.1316 - precision: 0.4153 - recall: 0.9800 - auc: 0.9847 - accuracy: 0.9444 - val_loss: 0.1739 - val_precision: 0.3200 - val_recall: 1.0000 - val_auc: 0.

...

12/12 [==============================] - 1s 61ms/step - loss: 0.2954 - precision: 0.4337 - recall: 0.9000 - auc: 0.9416 - accuracy: 0.8629 - val_loss: 0.3771 - val_precision: 0.4375 - val_recall: 0.6364 - val_auc: 0.8659 - val_accuracy: 0.8602
Epoch 12/100
12/12 [==============================] - 1s 68ms/step - loss: 0.2895 - precision: 0.4368 - recall: 0.9500 - auc: 0.9433 - accuracy: 0.8629 - val_loss: 0.3558 - val_precision: 0.4118 - val_recall: 0.6364 - val_auc: 0.8714 - val_accuracy: 0.8495
Epoch 13/100
12/12 [==============================] - 1s 59ms/step - loss: 0.2880 - precision: 0.4167 - recall: 0.8750 - auc: 0.9462 - accuracy: 0.8548 - val_loss: 0.3607 - val_precision: 0.3333 - val_recall: 0.5455 - val_auc: 0.8786 - val_accuracy: 0.8172
Epoch 14/100
12/12 [==============================] - 1s 58ms/step - loss: 0.2696 - precision: 0.4737 - recall: 0.9000 - auc: 0.9507 - accuracy: 0.8817 - val_loss: 0.3584 - val_precision: 0.3333 - val_recall: 0.5455 - val_auc: 0.8819 - val_ac

Epoch 43/100
12/12 [==============================] - 1s 58ms/step - loss: 0.1301 - precision: 0.6610 - recall: 0.9750 - auc: 0.9881 - accuracy: 0.9435 - val_loss: 0.1868 - val_precision: 0.7500 - val_recall: 0.8182 - val_auc: 0.9612 - val_accuracy: 0.9462
Epoch 44/100
12/12 [==============================] - 1s 60ms/step - loss: 0.1713 - precision: 0.6102 - recall: 0.9000 - auc: 0.9814 - accuracy: 0.9274 - val_loss: 0.1721 - val_precision: 0.7273 - val_recall: 0.7273 - val_auc: 0.9634 - val_accuracy: 0.9355
Epoch 45/100
12/12 [==============================] - 1s 56ms/step - loss: 0.1290 - precision: 0.6094 - recall: 0.9750 - auc: 0.9881 - accuracy: 0.9301 - val_loss: 0.1885 - val_precision: 0.6667 - val_recall: 0.7273 - val_auc: 0.9606 - val_accuracy: 0.9247
Epoch 46/100
12/12 [==============================] - 1s 56ms/step - loss: 0.1193 - precision: 0.6724 - recall: 0.9750 - auc: 0.9907 - accuracy: 0.9462 - val_loss: 0.1734 - val_precision: 0.6667 - val_recall: 0.7273 - val_auc: 0.

114/114 [==============================] - 7s 58ms/step - loss: 0.3112 - precision: 0.4437 - recall: 0.9139 - auc: 0.9365 - accuracy: 0.8467 - val_loss: 0.3512 - val_precision: 0.4509 - val_recall: 0.8417 - val_auc: 0.9095 - val_accuracy: 0.8440
Epoch 20/100
114/114 [==============================] - 6s 56ms/step - loss: 0.3025 - precision: 0.4676 - recall: 0.8918 - auc: 0.9393 - accuracy: 0.8602 - val_loss: 0.4184 - val_precision: 0.4015 - val_recall: 0.8667 - val_auc: 0.9102 - val_accuracy: 0.8121
Epoch 21/100
114/114 [==============================] - 6s 54ms/step - loss: 0.2841 - precision: 0.4646 - recall: 0.9117 - auc: 0.9486 - accuracy: 0.8582 - val_loss: 0.3431 - val_precision: 0.4773 - val_recall: 0.8750 - val_auc: 0.9200 - val_accuracy: 0.8571
Epoch 22/100
114/114 [==============================] - 6s 54ms/step - loss: 0.2658 - precision: 0.4860 - recall: 0.9183 - auc: 0.9534 - accuracy: 0.8690 - val_loss: 0.3369 - val_precision: 0.4703 - val_recall: 0.8583 - val_auc: 0.9164 

Epoch 51/100
114/114 [==============================] - 6s 55ms/step - loss: 0.1620 - precision: 0.6243 - recall: 0.9647 - auc: 0.9792 - accuracy: 0.9234 - val_loss: 0.2954 - val_precision: 0.5596 - val_recall: 0.9000 - val_auc: 0.9415 - val_accuracy: 0.8934
Epoch 52/100
29/29 [==============================] - 1s 18ms/step - loss: 0.2714 - precision: 0.5847 - recall: 0.8917 - auc: 0.9420 - accuracy: 0.9022
Loss: 0.27136439085006714, Precision: 0.5846994519233704, Recall: 0.8916666507720947, AUC: 0.9420410990715027, Accuracy: 0.9021977782249451
19 di 2581) Caricamento ticker CNDT                            [0.55507559 5.03921569]
Epoch 1/100
39/39 [==============================] - 23s 185ms/step - loss: 1.6018 - precision: 0.1439 - recall: 0.3578 - auc: 0.6044 - accuracy: 0.7551 - val_loss: 0.8163 - val_precision: 0.1875 - val_recall: 0.4091 - val_auc: 0.6823 - val_accuracy: 0.6634
Epoch 2/100
39/39 [==============================] - 3s 65ms/step - loss: 0.7193 - precision: 0.1693 - r

39/39 [==============================] - 2s 55ms/step - loss: 0.2007 - precision: 0.4700 - recall: 0.9358 - auc: 0.9685 - accuracy: 0.9011 - val_loss: 0.4216 - val_precision: 0.4545 - val_recall: 0.7955 - val_auc: 0.8874 - val_accuracy: 0.8350
Epoch 30/100
39/39 [==============================] - 2s 64ms/step - loss: 0.1966 - precision: 0.4496 - recall: 0.9817 - auc: 0.9672 - accuracy: 0.8921 - val_loss: 0.4148 - val_precision: 0.4545 - val_recall: 0.7955 - val_auc: 0.8687 - val_accuracy: 0.8350
Epoch 31/100
39/39 [==============================] - 2s 56ms/step - loss: 0.1843 - precision: 0.4882 - recall: 0.9450 - auc: 0.9713 - accuracy: 0.9075 - val_loss: 0.4154 - val_precision: 0.4416 - val_recall: 0.7727 - val_auc: 0.8913 - val_accuracy: 0.8285
Epoch 32/100
39/39 [==============================] - 2s 60ms/step - loss: 0.1717 - precision: 0.4929 - recall: 0.9541 - auc: 0.9753 - accuracy: 0.9092 - val_loss: 0.3744 - val_precision: 0.5072 - val_recall: 0.7955 - val_auc: 0.8900 - val_ac